# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

Variables of note:

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
'1000000' = 1.000000

FHINCP - income flag - 1: yes https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

GRNTP - gross rent (monthly) (not 0: n/a,not paying rent, owned)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GRNTP.json

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
from AMI_BANDS_2021 import *

In [4]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',
             '0400118','0400119','0400120','0400121','0400122','0400123',
             '0400125','0400128','0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,HHLDRRAC1P,HHLDRHISP'

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get data - build base df

In [6]:
df = get.get_puma(sample,y1,data_cols)

In [7]:
#make geo identifiers, reduce df to PHX PUMAs
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col

#clean data cols to be numeric
for col in df.columns[1:]: df[col] = df[col].astype(float)

#for PHX housing
df.to_excel('output/raw_Census_rent.xlsx',index=False)

In [8]:
#assign a household size
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,25],
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)

#generate adjusted household income using ACS adjustment factor
df['HINCP'] = df.ADJINC * df.HINCP

In [9]:
df['White_Alone']=np.where((df['HHLDRHISP']==1)&(df['HHLDRRAC1P']==1),'WhiteAlone','Other')

In [11]:
#identify the records that are renter households, but with no other info
#will be included in supply but not demand
bad_records = df[(df.HHSz.isna())&(df.HINCP<0)|(df.HINCP==0)&(df.GRNTP==0)].copy()
print(f'there are {bad_records.WGTP.sum()} households with no size,income, or rent information') 

there are 4045.0 households with no size,income, or rent information


In [12]:
#df for supply of units that are for rent - rent is not 0 
#(i.e., universe of renter households)
df_supply = df[(df.GRNTP!=0)].copy()

In [13]:
df_demand = df[~(df.HHSz.isna()) & (df.HINCP>-60000)&(df.HINCP!=0)&(df.GRNTP!=0)].copy()

In [14]:
dfs = [df_supply,df_demand]

In [15]:
#for both supply and demand dataframes, assign housing cost (rent), recategorize based on AMI calculation

for frame in dfs:
    frame['hou_cost'] = frame.GRNTP
    frame['aff_cost'] = np.where(frame.HINCP>=1,(frame.HINCP*0.333)/12,0)

    #who is the unit affordable to based on the rent
    frame['unit_aff'] = np.where(frame.HHSz=='1',pd.cut(frame['hou_cost'],bins=affordable_rent['1'],labels=inc_lbl),
                      np.where(frame.HHSz=='2',pd.cut(frame['hou_cost'],bins=affordable_rent['2'],labels=inc_lbl),
                      np.where(frame.HHSz=='3',pd.cut(frame['hou_cost'],bins=affordable_rent['3'],labels=inc_lbl),
                      np.where(frame.HHSz=='4',pd.cut(frame['hou_cost'],bins=affordable_rent['4'],labels=inc_lbl),
                      np.where(frame.HHSz=='5',pd.cut(frame['hou_cost'],bins=affordable_rent['5'],labels=inc_lbl),
                      np.where(frame.HHSz=='6',pd.cut(frame['hou_cost'],bins=affordable_rent['6'],labels=inc_lbl),
                      np.where(frame.HHSz=='7',pd.cut(frame['hou_cost'],bins=affordable_rent['7'],labels=inc_lbl),
                      np.where(frame.HHSz=='8',pd.cut(frame['hou_cost'],bins=affordable_rent['8'],labels=inc_lbl),''))))))))

In [16]:
#assign AMI range based on household size and ami dictionaries
df_demand['AMI_range'] = np.where((df_demand['HINCP']<=df_demand['HHSz'].map(AMI_30pct)),inc_lbl[0],
                   np.where((df_demand['HINCP']>df_demand['HHSz'].map(AMI_30pct))&(df_demand['HINCP']<=df_demand['HHSz'].map(AMI_50pct)),inc_lbl[1],
                   np.where((df_demand['HINCP']>df_demand['HHSz'].map(AMI_50pct))&(df_demand['HINCP']<=df_demand['HHSz'].map(AMI_80pct)),
                   inc_lbl[2],np.where((df_demand['HINCP']>df_demand['HHSz'].map(AMI_80pct))&(df_demand['HINCP']<=df_demand['HHSz'].map(AMI_100pct)),
                   inc_lbl[3],np.where((df_demand['HINCP']>df_demand['HHSz'].map(AMI_100pct))&(df_demand['HINCP']<=df_demand['HHSz'].map(AMI_120pct)),
                                       inc_lbl[4],inc_lbl[5])))))

#assign cost burden
df_demand['cost_burdened'] = np.where(df_demand.hou_cost>df_demand.aff_cost,'burdened','not burdened')

In [17]:
df_demand.head()

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,HHLDRRAC1P,HHLDRHISP,WGTP1,...,WGTP78,WGTP79,WGTP80,HHSz,White_Alone,hou_cost,aff_cost,unit_aff,AMI_range,cost_burdened
3612,0400120,177353.6016,3.0,42.0,1.029928,1000000.0,2460.0,1.0,1.0,74.0,...,44.0,43.0,42.0,3,WhiteAlone,2460.0,4921.562444,o120_ami,o120_ami,not burdened
3637,0400119,55307.1336,7.0,172.0,1.029928,1000000.0,1405.0,9.0,2.0,70.0,...,164.0,282.0,65.0,7,Other,1405.0,1534.772957,50_80_ami,50_80_ami,not burdened
3639,0400116,70344.0824,2.0,177.0,1.029928,1000000.0,1808.0,8.0,2.0,244.0,...,299.0,161.0,194.0,2,Other,1808.0,1952.048287,100_120_ami,100_120_ami,not burdened
3642,0400128,9887.3088,1.0,114.0,1.029928,1000000.0,1168.0,1.0,1.0,98.0,...,224.0,33.0,130.0,1,WhiteAlone,1168.0,274.372819,80_100_ami,u30_ami,burdened
3645,0400120,28065.5380,4.0,73.0,1.029928,1000000.0,1450.0,1.0,1.0,73.0,...,76.0,121.0,70.0,4,WhiteAlone,1450.0,778.818680,50_80_ami,30_50_ami,burdened


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [18]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [19]:
drop_cols_supply = ['HINCP','NP','ADJINC','ADJHSG','GRNTP','hou_cost', 'HHSz','HHLDRHISP','HHLDRRAC1P']

drop_cols_demand = drop_cols_supply + ['aff_cost']

##### demand tables

In [20]:
df_d = df_demand.copy().drop(columns=drop_cols_demand)
df_d = df_d.groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened','White_Alone']).sum().reset_index()

In [21]:
rent = df_d.copy().groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened','White_Alone']).sum().reset_index()
rent = make_est(rent)
rent = rent.drop(columns=repwts)

In [22]:
rent_ami = df_d.copy().drop(columns=['unit_aff','cost_burdened']).groupby(['GEO_ID','AMI_range','White_Alone']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)

In [23]:
rent_aff = df_d.copy().drop(columns=['AMI_range','cost_burdened']).groupby(['GEO_ID','unit_aff','White_Alone']).sum().reset_index()
rent_aff = make_est(rent_aff)
rent_aff = rent_aff.drop(columns=repwts)

In [24]:
cost_burd = df_d.copy().drop(columns=['unit_aff','AMI_range']).groupby(['GEO_ID','cost_burdened','White_Alone']).sum().reset_index()
cost_burd = make_est(cost_burd)
cost_burd = cost_burd.drop(columns=repwts)

In [25]:
cost_burd_ami = df_d.copy().drop(columns=['unit_aff']).groupby(['GEO_ID','cost_burdened','AMI_range','White_Alone']).sum().reset_index()
cost_burd_ami = make_est(cost_burd_ami)
cost_burd_ami = cost_burd_ami.drop(columns=repwts)

In [26]:
ami_aff = df_d.copy().drop(columns='cost_burdened').groupby(['GEO_ID','AMI_range','unit_aff','White_Alone']).sum().reset_index()
ami_aff = make_est(ami_aff)
ami_aff = ami_aff.drop(columns=repwts)

In [27]:
rent = pd.pivot_table(rent,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff','cost_burdened','White_Alone'],aggfunc=np.sum).reset_index()

rent_ami = pd.pivot_table(rent_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','White_Alone'],aggfunc=np.sum).reset_index()

rent_aff = pd.pivot_table(rent_aff,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff','White_Alone'],aggfunc=np.sum).reset_index()

cost_burd = pd.pivot_table(cost_burd,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['cost_burdened','White_Alone'],aggfunc=np.sum).reset_index()

cost_burd_ami = pd.pivot_table(cost_burd_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['cost_burdened','AMI_range','White_Alone'],aggfunc=np.sum).reset_index()

ami_aff = pd.pivot_table(ami_aff,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff','White_Alone'],aggfunc=np.sum).reset_index()

##### supply table

In [28]:
df_s = df_supply.copy().drop(columns=drop_cols_supply)
df_s = df_s.groupby(['GEO_ID','unit_aff']).sum().reset_index()

In [29]:
supply_aff = df_s.copy().groupby(['GEO_ID','unit_aff']).sum().reset_index()
supply_aff = make_est(supply_aff)
supply_aff = supply_aff.drop(columns=repwts)

In [30]:
supply_aff = pd.pivot_table(supply_aff,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

In [31]:
with pd.ExcelWriter(f'../../output/pums_gap_rent/pums_rent_gap_whitenonwhite_033124.xlsx') as writer:
    rent.to_excel(writer,sheet_name='full_renter')
    rent_ami.to_excel(writer,sheet_name='hh_ami_lvl_renter')
    rent_aff.to_excel(writer,sheet_name='hh_afford_byAMI_renter')
    cost_burd.to_excel(writer,sheet_name='cost_burdened_renter')
    cost_burd_ami.to_excel(writer,sheet_name='cost_burdened_byAMI_renter')
    ami_aff.to_excel(writer,sheet_name='demand_supply_amilvl')
    supply_aff.to_excel(writer,sheet_name='supply_ami_aff')